import pandas!

In [ ]:
import pandas as pd

Load all the dataframes

In [ ]:
suppliers = pd.read_csv("https://www.dropbox.com/s/jjbacrux6bc1b4s/suppliers.csv?dl=1")
demand = pd.read_csv("https://www.dropbox.com/s/0yikq3pugq87vl9/demand_forecasts.csv?dl=1")
recipes = pd.read_csv("https://www.dropbox.com/s/az97voeh6i8x3i2/recipes.csv?dl=1")

In [ ]:
suppliers

In [ ]:
demand

In [ ]:
recipes

Choose suppliers for each ingredient for each product.
Select the cheapest option, and the least tax option, and the fastest option.


In [ ]:
cheapest_supplier = suppliers.loc[suppliers.groupby('ingredient_uuid')['cost_per_unit'].idxmin()]
cheapest_supplier

In [ ]:
suppliers.loc[suppliers.groupby('ingredient_uuid')['total_tax_per_unit'].idxmin()]

In [ ]:
suppliers.loc[suppliers.groupby('ingredient_uuid')['time_to_deliver'].idxmin()]

Work out total demand for each ingredient, using the demand for each product by month, and over a year.

In [ ]:
ingredient_demand = demand.merge(recipes, on='rec_id')
ingredient_demand['ingredient_demand'] = ingredient_demand['demand']*ingredient_demand['qty']
ingredient_demand

In [ ]:
monthly_ingredient_demand = ingredient_demand.groupby(['ingredient_uuid','month'])[['ingredient_demand']].sum()
monthly_ingredient_demand

In [ ]:
monthly_ingredient_demand.groupby('ingredient_uuid').plot(figsize=(10,10))

In [ ]:
ingredient_demand.merge(cheapest_supplier, on='ingredient_uuid')
ingredient_demand['ingredient_demand'] = ingredient_demand['demand']*ingredient_demand['qty']
ingredient_demand

In [ ]:
demand_by_supply = ingredient_demand.merge(cheapest_supplier, on='ingredient_uuid')
demand_by_supply['ingredient_cost'] = demand_by_supply['ingredient_demand']*demand_by_supply['cost_per_unit']
demand_by_supply

Work out total cost per unit to product


In [ ]:
demand_by_supply.groupby('rec_id')['ingredient_cost'].sum()

In [ ]:
demand_by_supply.groupby('rec_id')['ingredient_cost'].sum().plot.bar()

(Homework) work out how to find the total cost of goods needed for each producer country.


In [ ]:
ingredient_demand.merge(cheapest_supplier, on="ingredient_uuid").groupby('country_of_origin').sum()

(Homework) work out how to find the total tax, using the cheapest supplier for each ingredient, to create 1000 of each recipe.

In [ ]:
q = ingredient_demand.merge(cheapest_supplier, on="ingredient_uuid").groupby('ingredient_uuid').sum()
q[['total_tax_per_unit']]

In [ ]:
q['total_tax_per_unit'].plot.bar(x = "ingredient_uuid", y="total_tax_per_unit")

In [ ]:
q['total_cost'] = q['ingredient_demand']*q['cost_per_unit'] + q['ingredient_demand']*q['total_tax_per_unit']
q

In [ ]:
q['total_cost'].plot.bar()

In [ ]:
q[['total_cost', 'time_to_deliver']].plot.scatter(x='total_cost', y='time_to_deliver')

In [ ]:
q = ingredient_demand.merge(cheapest_supplier, on="ingredient_uuid").groupby('ingredient_uuid')[['total_tax_per_unit']].sum()
q

In [ ]:
#Day 5

In [ ]:
from random import randint
import numpy as np

In [ ]:
df = pd.read_csv("https://www.dropbox.com/s/vz1l18qa9xxvhy3/demand_trend.csv?dl=1", index_col=0)

df['date'] = pd.to_datetime(df['date'])
q = df[ ['date', 'demand'] ]
q.plot(x='date', y='demand', figsize=(10,10))

In [ ]:
q

In [ ]:
q.dropna()

In [ ]:
df.fillna( df['demand'].mean() )

In [ ]:
df.fillna( df['demand'].mean() )['demand'].plot()

In [ ]:
df.fillna( method='ffill' )

In [ ]:
df.fillna( method='bfill' )

In [ ]:
df.fillna( method='ffill' ).fillna( method='bfill' )

In [ ]:
df.fillna( method='ffill' ).fillna( method='bfill' )['demand'].plot()

In [ ]:
df['day_of_year'] = df['date'].apply( lambda x : x.dayofyear )

df['week_of_year'] = df['date'].apply( lambda x : x.weekofyear )

df['day_of_week'] = df['date'].apply( lambda x : x.day_of_week )

df['is_month_start'] = df['date'].apply( lambda x : x.is_month_start )

df['month'] = df['date'].apply( lambda x : x.month )

In [ ]:
df

In [ ]:
df.fillna(df['demand'].mean()).groupby( 'week_of_year' )['demand'].sum().plot()

In [ ]:
df.fillna(df['demand'].mean()).groupby( 'day_of_week' )['demand'].sum().plot.bar()

In [ ]:
#homework : create a bar chart grouping all the data by month. Is any one month significantly higher than the others

In [ ]:
#homework (research needed) : use diff + shift functions to work out rate of demand growth month/month and year / year
#take care to notice what happens with nan

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

In [ ]:
cal = calendar()
holidays = cal.holidays(start=df['date'].min(), end=df['date'].max())

df['holiday'] = df['date'].isin(holidays)

In [ ]:
df.groupby('holiday')['demand'].mean().plot.bar()

In [ ]:
df['diff_day'] = df.fillna( method='ffill' ).fillna( method='bfill' )['demand'].diff(periods=1)


In [ ]:
df.fillna( method='ffill' ).fillna( method='bfill' )['demand'].diff(periods=10)

In [ ]:
df

In [ ]:
df['demand_shift'] = df['demand'].shift(2)
df

In [ ]:
df['dod_demand'] = df['demand'] / df['demand_shift']
df

In [ ]:
#using the index

In [ ]:
%%time
df[df['date'] == pd.Timestamp("2020-01-01")]

In [ ]:
df2 = df.set_index("date")

In [ ]:
df2

In [ ]:
%%time
df2[df2.index == pd.Timestamp("2020-01-01")]

In [ ]:
dr = pd.date_range(start="01-01-1700", end="01-01-2200", freq="h")
df3 = pd.DataFrame(np.random.randn(len(dr), 1), columns=['mycol'])
df3['date'] = dr

In [ ]:
df3

In [ ]:
%%time
df3[df3['date'] == pd.Timestamp("2020-01-01")]

In [ ]:
df4 = df3.set_index('date')
df4

In [ ]:
%%time
df4[df4.index == pd.Timestamp("2020-01-01")]

In [ ]:
%%time
df4.loc[pd.Timestamp("2020-01-01")]

In [ ]:
df5 = suppliers.set_index(['ingredient_uuid', 'supp_id'])

In [ ]:
df5

In [ ]:
%%time
df5[ df5.index == ('vanilla','megacorp') ]

In [ ]:
df5.loc[ ('vanilla','megacorp') ]

In [ ]:
df5.xs("vanilla", level="ingredient_uuid")

In [ ]:
df5.xs("acme_inc", level="supp_id")

In [ ]:
df5.xs("acme_inc", level="supp_id")

In [ ]:
#this is the "real deal"
#select no slice (so everything) for index col one, then one value from index col two. ", :" means, select all cols

df5.loc[(slice(None), "omni_inc"), :]

In [ ]:
df5.loc[(slice(None), "omni_inc"), "unit_name"]

In [ ]:
df5.loc[(slice(None), "omni_inc"), ["unit_name", "cost_per_unit"] ]

In [ ]:
#as a last resort, cos it loses the speed-up of the index. Just don't.
df5[df5.index.get_level_values(1) == "omni_inc"]

In [ ]:
idemand = demand.set_index('rec_id')
irecipes = recipes.set_index('rec_id')

#only possible in most recent versions of pandas ;) so take care with that one
iingredient_demand = idemand.merge(irecipes, on='rec_id')


In [ ]:
iingredient_demand

In [ ]:
iingredient_demand = idemand.merge(irecipes, left_index=True, right_index=True)


In [ ]:
iingredient_demand

In [ ]:
#going hardcore
idx = pd.IndexSlice
df5.loc[ idx[ ['milk_powder', 'hazelnut'], : ] ]

In [ ]:
df6 = df5.reset_index().set_index( ['ingredient_uuid', 'cost_per_unit'] ) #needed first unfortunately

idx = pd.IndexSlice
df6.loc[ idx[ ['milk_powder', 'hazelnut'], : ] ]

In [ ]:
df6=df6.sort_index()

In [ ]:
i = idx[ :, slice(2500, 4000) , : ]
df6.loc[ i ]

In [ ]:
#or in one step once you master it
#this is the best solution for most complex cases
#the final ":" selects a set of cols in a similar way

df6.loc[ idx[ :, slice(2500, 4000) , : ] ]